In [26]:
# -*- coding: utf-8 -*-

u"""
与えられた文書からマルコフ連鎖のためのチェーン（連鎖）を作成して、DBに保存するファイル
"""

import unittest

import re
import MeCab
import sqlite3
from collections import defaultdict


class PrepareChain(object):
    """
    チェーンを作成してDBに保存するクラス
    """

    BEGIN = "__BEGIN_SENTENCE__"
    END = "__END_SENTENCE__"

    DB_PATH = "chain-test.db"
    DB_SCHEMA_PATH = "schema.sql"

    def __init__(self, text):
        u"""
        初期化メソッド
        @param text Chainを生成するための文章
        """
        self.text = text

        # 形態素解析用タガー
        self.tagger = MeCab.Tagger('-Ochasen')

    def make_triplet_freqs(self):
        u"""
        形態素解析から3つ組の出現回数まで
        @return 3つ組とその出現回数の辞書 key: 3つ組（タプル） val: 出現回数
        """
        # 長い文章をセンテンス毎に分割(_divideは後述)
        sentences = self._divide(self.text)

        # 3つ組の出現回数(defaultdictで辞書の初期化→出現回数の数え上げに最適)
        triplet_freqs = defaultdict(int)

        # センテンス毎に3つ組にする
        for sentence in sentences:
            # 形態素解析(_morphologizal_analysisは後述)
            morphemes = self._morphological_analysis(sentence)
            # 3つ組をつくる
            triplets = self._make_triplet(morphemes)
            # 出現回数を加算
            for (triplet, n) in triplets.items():
                triplet_freqs[triplet] += n

        return triplet_freqs
    
    #ここから手段
    def _divide(self, text):
        """
        「。」や改行などで区切られた長い文章を一文ずつに分ける
        @param text 分割前の文章
        @return 一文ずつの配列
        """
        # 改行文字以外の分割文字（正規表現表記）
        delimiter = r'。|．|\.'

        # 全ての分割文字を改行文字に置換（splitしたときに「。」などの情報を無くさないため）
        text = re.sub(delimiter, '\n', text)
        

        # 改行文字で分割
        sentences = text.splitlines()

        # 前後の空白文字を削除
        sentences = [sentence.strip() for sentence in sentences]
        return sentences

    def _morphological_analysis(self, sentence):
        """
        一文を形態素解析する
        @param sentence 一文
        @return 形態素で分割された配列
        """
        morphemes = []
        node = self.tagger.parseToNode(sentence)
        while node:
            if node.posid != 0:
                morpheme = node.surface
                morphemes.append(morpheme)
            node = node.next
        return morphemes

    def _make_triplet(self, morphemes):
        """
        形態素解析で分割された配列を、形態素毎に3つ組にしてその出現回数を数える
        @param morphemes 形態素配列
        @return 3つ組とその出現回数の辞書 key: 3つ組（タプル） val: 出現回数
        """
        # 3つ組をつくれない場合は終える
        if len(morphemes) < 3:
            return {}

        # 出現回数の辞書
        triplet_freqs = defaultdict(int)

        # 繰り返し
        for i in range(len(morphemes)-2):
            triplet = tuple(morphemes[i:i+3])
            triplet_freqs[triplet] += 1

        # beginを追加、要するに<BOS>
        triplet = (PrepareChain.BEGIN, morphemes[0], morphemes[1])
        triplet_freqs[triplet] = 1#あたりまえだけど一回

        # endを追加、要するに<EOS>
        triplet = (morphemes[-2], morphemes[-1], PrepareChain.END)
        triplet_freqs[triplet] = 1

        return triplet_freqs

    def save(self, triplet_freqs, init=False):
        """
        3つ組毎に出現回数をDBに保存
        @param triplet_freqs 3つ組とその出現回数の辞書 key: 3つ組（タプル） val: 出現回数
        """
        # DBオープン。ない場合は新規生成される
        con = sqlite3.connect(PrepareChain.DB_PATH)

        # 初期化から始める場合
        if init:
            # DBの初期化
            #スキーマ(SQL)から読み取り
            with open(PrepareChain.DB_SCHEMA_PATH, "r") as f:
                schema = f.read()
                con.executescript(schema)

            # データ整形。[triplet*3 ,出現回数]
            datas = [(triplet[0], triplet[1], triplet[2], freq) for (triplet, freq) in triplet_freqs.items()]
            #print(datas)
            # データ挿入
            p_statement = "insert into chain_freqs (prefix1, prefix2, suffix, freq) values (?, ?, ?, ?)"
            con.executemany(p_statement, datas)#テーブル作成

        # コミットしてクローズ
        con.commit()
        con.close()

    def show(self, triplet_freqs):
        """
        3つ組毎の出現回数を出力する
        @param triplet_freqs 3つ組とその出現回数の辞書 key: 3つ組（タプル） val: 出現回数
        """
        for triplet in triplet_freqs:
            print("|".join(triplet), "\t", triplet_freqs[triplet])

def get_object(path):
    '''pathからオブジェクト(テキスト)生成
    @param path ファイルパス(相対)
    return object テキスト'''
    f = open(path)
    text = f.read()
    text = re.sub('<EOS>|<BOS>','',text)
    f.close()
    return text
    
if __name__ == '__main__':
    text = get_object('question_extract/takken0.txt')
    chain = PrepareChain(text)
    triplet_freqs = chain.make_triplet_freqs()
    chain.save(triplet_freqs, True)
    chain.show(triplet_freqs)

|| 	 1
||の 	 1
|の|うち 	 1
の|うち|、 	 649
うち|、|宅地 	 147
、|宅地|建物 	 199
宅地|建物|取引 	 499
建物|取引|業法 	 228
取引|業法|の 	 124
業法|の|免許 	 2
の|免許|を 	 3
免許|を|受ける 	 1
を|受ける|必要 	 5
受ける|必要|の 	 5
必要|の|ない 	 2
の|ない|もの 	 2
ない|もの|は 	 28
もの|は|どれ 	 641
は|どれ|か 	 653
__BEGIN_SENTENCE__|| 	 1
どれ|か|__END_SENTENCE__ 	 656
A|（|個人 	 1
（|個人|） 	 4
個人|）|の 	 1
）|の|宅地 	 1
の|宅地|建物 	 5
の|免許|（ 	 19
免許|（|以下 	 19
（|以下|この 	 179
以下|この|問 	 179
この|問|において 	 199
問|において|「 	 185
において|「|免許 	 18
「|免許|」 	 19
免許|」|という 	 19
__BEGIN_SENTENCE__|A|（ 	 1
」|という|__END_SENTENCE__ 	 179
）|に関する|次 	 103
に関する|次|の 	 532
次|の|記述 	 653
の|記述|の 	 661
記述|の|うち 	 661
うち|、|正しい 	 171
、|正しい|もの 	 387
正しい|もの|は 	 400
__BEGIN_SENTENCE__|）|に関する 	 100
建物|取引|業 	 33
取引|業|の 	 18
業|の|免許 	 18
__BEGIN_SENTENCE__|宅地|建物 	 233
業法|の|規定 	 122
の|規定|に 	 273
規定|に|よれ 	 209
に|よれ|ば 	 341
よれ|ば|、 	 337
ば|、|正しい 	 210
もの|は|いくつ 	 27
は|いくつ|ある 	 27
いくつ|ある|か 	 28
ある|か|__END_SENTENCE__ 	 28
建物|取引|業者 	 223
取引|業者|A 	 54
業者|A|社 	 14
A|社|が 	 6
社|が|行う 	 2
が|行う|業務 	 8
行う|業務|に関する 	 8
業務|に関する|次 	 10
規定|に|違